In [1]:
LOGINFO = 1
def log_info(*args):
    if LOGINFO:
        for i in args:
            print(i,end=' ')
        print()

In [2]:
import os
import numpy as np
import cv2

def input_data(npz=True):
    if npz:
        bird_data = np.load('bird_data.npz')
        return bird_data['train_img'],bird_data['test_img'],bird_data['train_label'],bird_data['test_label']
    else:      
        data_path = os.path.join('..','data','CUB_200_2011')
        log_info(os.listdir(data_path))

        train_test_split_file = os.path.join(data_path,'train_test_split.txt')
        with open(train_test_split_file,'r') as file:
            train_test_split = np.array([i.split()[1] for i in file.readlines()]).astype('bool')
        log_info(train_test_split,train_test_split.size)

        img_paths_file = os.path.join(data_path,'images.txt')
        with open(img_paths_file,'r') as file:
            img_paths = [i.split()[1] for i in file.readlines()]
        log_info(img_paths[:1],len(img_paths))

        img_labels_file = os.path.join(data_path,'image_class_labels.txt')
        with open(img_labels_file,'r') as file:
            img_labels = np.array([i.split()[1] for i in file.readlines()]).astype('int')
        log_info(img_labels,len(img_labels))

        img_dir = os.path.join(data_path,'images')

        img_paths_train = [os.path.join(img_dir,os.path.sep.join(path.split('/'))) for i,path in enumerate(img_paths) if train_test_split[i]]
        log_info(img_paths_train[:1],len(img_paths_train))
        img_paths_test = [os.path.join(img_dir,os.path.sep.join(path.split('/'))) for i,path in enumerate(img_paths) if not train_test_split[i]]
        log_info(img_paths_test[:1],len(img_paths_test))

        train_img = np.array([cv2.resize(cv2.imread(i),(64,64)) for i in img_paths_train])
        test_img = np.array([cv2.resize(cv2.imread(i),(64,64)) for i in img_paths_test])
        train_label = np.array([l for i,l in enumerate(img_labels) if train_test_split[i] ])
        test_label = np.array([l for i,l in enumerate(img_labels) if not train_test_split[i]])
        log_info(train_label,train_label.size)
        log_info(test_label,test_label.size)

        np.savez('bird_data',train_img=train_img,test_img=test_img,train_label=train_label,test_label=test_label)
        return train_img,test_img,train_label,test_label

In [3]:
train_img,test_img,train_label,test_label = input_data()
log_info('type:',type(train_img),type(train_label))
log_info('shape:',train_img.shape,train_label.shape)
log_info('size:',train_img.size,train_label.size)

type: <class 'numpy.ndarray'> <class 'numpy.ndarray'> 
shape: (5994, 64, 64, 3) (5994,) 
size: 73654272 5994 


In [4]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

num_classes = 200
model_name = 'bird.h5'

# 数据预处理，把 0-255的灰度值转成 0-1 之间的浮点数
x_train = train_img.astype('float32')/255
x_test = test_img.astype('float32')/255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(train_label-1, num_classes)
y_test = keras.utils.to_categorical(test_label-1, num_classes)
log_info('type:',type(x_train),type(y_train))
log_info('shape:',x_train.shape,y_train.shape)
log_info('size:',x_train.size,y_train.size)

# shuffle
x_train, y_train = np.array(x_train),np.array(y_train)
index = [i for i in range(len(y_train))]
np.random.shuffle(index)
x_train = x_train[index]
y_train = y_train[index]

# 拆分验证集
(x_valid, x_train) = x_train[5000:], x_train[:5000] # 994+5000
(y_valid, y_train) = y_train[5000:], y_train[:5000]

Using TensorFlow backend.


type: <class 'numpy.ndarray'> <class 'numpy.ndarray'> 
shape: (5994, 64, 64, 3) (5994, 200) 
size: 73654272 1198800 


In [5]:
from keras.preprocessing.image import ImageDataGenerator
 
datagen_train = ImageDataGenerator(
width_shift_range = 0.1,
height_shift_range = 0.1,
horizontal_flip = True)
 
datagen_train.fit(x_train)

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

# initiate optimizer
opt = keras.optimizers.adam(lr=0.001, decay=1e-6)

# train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
__________

In [7]:
# hist = model.fit(x_train, y_train, epochs=30, shuffle=True)
# model.save(model_name)
from keras.callbacks import ModelCheckpoint
 
batch_size=32
checkpoint = ModelCheckpoint(filepath='bird.weights.best.hdf5', verbose=1, save_best_only=True)
hist = model.fit_generator(datagen_train.flow(x_train, y_train, batch_size=32),
                steps_per_epoch=x_train.shape[0] // batch_size,
                epochs = 30,
                shuffle=True,
                verbose=1,
                callbacks=[checkpoint],
                validation_data=(x_valid, y_valid),
                validation_steps=x_valid.shape[0] // batch_size)

Epoch 1/30
156/156 [==============================] - 12s 74ms/step - loss: 5.3067 - acc: 0.0028 - val_loss: 5.3071 - val_acc: 0.0010

Epoch 00001: val_loss improved from inf to 5.30711, saving model to bird.weights.best.hdf5
Epoch 2/30
156/156 [==============================] - 8s 49ms/step - loss: 5.2967 - acc: 0.0060 - val_loss: 5.2727 - val_acc: 0.0050

Epoch 00002: val_loss improved from 5.30711 to 5.27266, saving model to bird.weights.best.hdf5
Epoch 3/30
156/156 [==============================] - 8s 49ms/step - loss: 5.1866 - acc: 0.0080 - val_loss: 5.1030 - val_acc: 0.0050

Epoch 00003: val_loss improved from 5.27266 to 5.10302, saving model to bird.weights.best.hdf5
Epoch 4/30
156/156 [==============================] - 8s 49ms/step - loss: 4.9959 - acc: 0.0142 - val_loss: 4.9295 - val_acc: 0.0171

Epoch 00004: val_loss improved from 5.10302 to 4.92945, saving model to bird.weights.best.hdf5
Epoch 5/30
156/156 [==============================] - 8s 49ms/step - loss: 4.8893 - acc

KeyboardInterrupt: 

In [ ]:
# evaluate
loss, accuracy = model.evaluate(x_test, y_test)
print('evaluate: loss:{} acc:{}'.format(loss, accuracy))


In [ ]:
for i in train_img:
    cv2.imshow('img',i)
    cv2.waitKey(0)
cv2.destroyAllWindows()